In [ ]:
%pip install packaging requests beautifulsoup4

In [3]:
import json
import os
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse

def get_html_with_requests(url):
    # Extrair domínio e caminho da URL
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path

    headers = {
        ':authority': domain,
        ':method': 'GET',
        ':path': path,
        ':scheme': 'https',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,es;q=0.5',
        'Cookie': 'ajs_anonymous_id=a03ba2e0-6658-4732-bca0-5c82a729610d; blueID=158f8c36-f884-4eeb-9dd2-41ddb36954f2; _fbp=fb.2.1719565497536.945089496421286543; _tt_enable_cookie=1; _ttp=yDIqAjbcrTkStMgfYKWUtQqV7qI; _scid=fe8f87fb-9164-4128-931f-6db8349a5902; _pin_unauth=dWlkPU16QmxZV015TVRndE16QmhOUzAwWmpRekxUZzROakF0Wm1VeFlUSTROREE1WXpJeg; _hjSessionUser_257734=eyJpZCI6IjZlYmVkNTczLWRlNDItNTljNy1iYjBkLWRjMzQxYzg1ZThkZiIsImNyZWF0ZWQiOjE3MTk1NjU0OTc1ODksImV4aXN0aW5nIjp0cnVlfQ==; __utmz=other; blueULC=blue; OptanonAlertBoxClosed=2024-06-28T10:07:39.896Z; _gac_UA-58792402-1=1.1719578340.CjwKCAjwvvmzBhA2EiwAtHVrbx5PuZ-Y6ZmDRgxSQAE3bhfdMkfQ6qO_h_InopnIh9t314DXR2UOdhoCJPMQAvD_BwE; origem=adwords; _rtbhouse_source_=AdWords; anonymousClient=true; _gcl_aw=GCL.1719608537.CjwKCAjwvvmzBhA2EiwAtHVrb7a4Gf1uUaMKzAcQ623DOKsXA0ynRueURMbfhKPxXrTvmq46s3W5ZBoCXV8QAvD_BwE; _gcl_gs=2.1.k1$i1719608535; _gid=GA1.3.1821486782.1720470235; _ScCbts=%5B%5D; petzCarrinho=true; JSESSIONID=9E56A4F6251336BC2EEC9640328DF1CE; _hjSession_257734=eyJpZCI6ImRmYjFhNzkxLTcwMDAtNDc5ZC1hZDA2LTE4NGYwZGMwOTQ3MSIsImMiOjE3MjA2MzEyODk1MzEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; OptanonConsent=isGpcEnabled=0&datestamp=Wed+Jul+10+2024+14%3A33%3A28+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=202404.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=5d44d058-e0ca-4634-af6a-6ae970855f35&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A1&AwaitingReconsent=false&intType=1&geolocation=BR%3BRJ; AWSALB=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; AWSALBCORS=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; _ga_JB2JHD7FCJ=GS1.1.1720631306.23.1.1720632809.57.0.0; _ga=GA1.1.604824160.1719565115; _uetsid=006ad4a03d6811ef82e9db6f4cebf44a; _uetvid=7e097870352d11ef8996917c57f63d87; _scid_r=fe8f87fb-9164-4128-931f-6db8349a5902; _derived_epik=dj0yJnU9OVpXTXl0T0xlTmRISjQ3aGwtVFQ1SWZjVmJuT0JiaFkmbj1zc3Z6NkNFQXNmdnZHWmFDdEpIX093Jm09MSZ0PUFBQUFBR2FPeGVzJnJtPTEmcnQ9QUFBQUFHYU94ZXMmc3A9Mg; cto_bundle=Eb4Xml9POEdtRUtaTE12dTVMYjVqT2xtMDIxaDE5blJXQWRHRkZXVEJlcUdNOTdFRGNwdGtMeWslMkZGVnBENzMwM0JYMDVmQkdIeVQ4NW1qMW9laEslMkZRQnl3dzElMkZPWFUlMkZmTFJxNGdLeXUlMkZTMTRqZyUyRndXSFZ4dEsxS1hyMk1ZeGpOQVU4bmU1bkFZY1lwcHp2M3NVZ1FWQWNmYmRYS0RIc1N1ZzFBcWRZcWNlOVcxRkVaeVZGd0YzQUxFMWglMkZheXhJa0cyJTJGdUg0NiUyQk92VCUyRnVZZmtxQWN6QVAlMkZzdyUzRCUzRA',
        'Priority': 'u=0, i',
        'Referer': f"{domain}/marcas",
        'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }

    # Remover cabeçalhos pseudo para requests
    headers = {k: v for k, v in headers.items() if not k.startswith(':')}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Erro ao obter HTML com requests: {response.status_code}")
        return None


def extract_product_details(product_url, last_sku):
    parsed_url = urlparse(product_url)
    domain = parsed_url.netloc
    path = parsed_url.path
    print(f"Extraindo detalhes do produto da URL: {product_url}")
    html_content = get_html_with_requests(product_url)
    if not html_content:
        print(f"Falha ao obter HTML para o produto: {product_url}")
        return (None, None, None, None, None, None, None), last_sku
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Criar pasta para imagens se não existir
    if not os.path.exists('imagens'):
        os.makedirs('imagens')
        print("Pasta 'imagens' criada.")
    
    # Extrair categoria principal e subcategorias
    breadcrumb = soup.find('nav', {'aria-label': 'breadcrumb'})
    categories = []
    if breadcrumb:
        breadcrumb_items = breadcrumb.find_all('span', class_='breadcrumb-item')
        if breadcrumb_items:
            main_category = breadcrumb_items[0].find('meta', itemprop='name')['content']
            subcategories = [item.find('meta', itemprop='name')['content'] for item in breadcrumb_items[1:]]
            categories = {
                "name": main_category,
                "link": "",  # Link não disponível
                "subcategories": [{"name": sub, "link": ""} for sub in subcategories]
            }
            print(f"Categorias encontradas: {categories}")
        else:
            print("Nenhuma categoria encontrada no breadcrumb.")
    else:
        print("Breadcrumb não encontrado.")
    
    # Extrair código de barras
    barcode_element = soup.find('span', {'id': 'product-code'})
    if barcode_element:
        barcode = barcode_element.text.replace('Código: ', '').strip()
        print(f"Código de barras encontrado: {barcode}")
    else:
        barcode = None
        print("Código de barras não encontrado.")
    
    # Extrair marca principal e submarca
    brand_element = soup.find('a', {'itemprop': 'brand'})
    if brand_element:
        brand = brand_element.find('span', {'itemprop': 'name'}).text.strip()
        print(f"Marca encontrada: {brand}")
    else:
        brand = None
        print("Marca não encontrada.")
    
    subbrand_element = soup.find('a', {'href': '/4-groomer'})
    if subbrand_element:
        subbrand = subbrand_element.find('span', class_='blue').text.strip()
        print(f"Submarca encontrada: {subbrand}")
    else:
        subbrand = None
    
  # Extrair descrição do produto
    description_element = soup.find('section', {'class': 'description', 'id': 'description'})
    if description_element:
        description = description_element.find('div', class_='spec-content').text.strip()
        print(f"Descrição encontrada: {description}")
    else:
        description = None
        print("Descrição não encontrada.")
    
    # Extrair especificações do produto
    specifications = []
    specifications_container = soup.find('div', class_='container')
    if specifications_container:
        spec_items = specifications_container.find_all('li', class_='specifications')
        for item in spec_items:
            spec_key = item.find('span', class_='spec-key').text.strip()
            spec_value = item.find('span', class_='spec-value').text.strip()
            specifications.append({
                'key': spec_key,
                'value': spec_value
            })
        print(f"Especificações encontradas: {specifications}")
    else:
        print("Especificações não encontradas.")
    
     # Extrair variações do produto
    variations = []
    variations_container = soup.find('div', {'id': 'popupVariacoes'})
    if variations_container:
        print("Container de variações encontrado.")
        print(f"HTML das variações:\n{variations_container.prettify()}\n")
        for variation in variations_container.find_all('div', class_='variacao-item'):
            variation_name = variation.find('div', class_='item-name').text.strip()
            price = variation.find('div', class_='modal-item-price').find('b').text.strip()
            price = re.sub(r'[^\d]', '', price)  # Remover caracteres não numéricos
            barcode_variation = variation['data-code']
            link_variation = domain + variation['data-urlvariacao']
            last_sku += 1
            print(f"Extraindo detalhes da variação: {variation_name}, Preço: {price}, Código de barras: {barcode_variation}, Link: {link_variation}, SKU: {last_sku}")
            
            variations.append({
                'name': variation_name,
                'price': price,
                'barcode': barcode_variation,
                'link': link_variation,
                'sku': last_sku
            })
        print(f"Variações encontradas: {variations}")
    else:
        print("Nenhuma variação encontrada.")
        # Verificar se o preço está diretamente no HTML
        price_element = soup.find('div', class_='current-price-left')
        if price_element:
            price = price_element.find('strong').text.strip()
            price = re.sub(r'[^\d]', '', price)  # Remover caracteres não numéricos
            last_sku += 1
            print(f"Preço encontrado: {price}, SKU: {last_sku}")
            
            variations.append({
                'name': 'default',
                'price': price,
                'barcode': barcode,
                'link': product_url,
                'sku': last_sku
            })
        else:
            print("Preço não encontrado.")
    
    
    product_details = {
        'categories': categories,
        'barcode': barcode,
        'brand': {
            'name': brand,
            'subbrand': {
                'name': subbrand
            } if subbrand else None
        },
        'description': description,
        'specifications': specifications,
        'variations': variations
    }
    
    print(f"Detalhes do produto extraídos: {product_details}")
    return product_details, last_sku

def extract_products(html_content, marca_name, marca_link, last_sku):
    soup = BeautifulSoup(html_content, 'html.parser')
    product_elements = soup.select('li.card-product.card-product-showcase[itemtype="http://schema.org/Product"]')
    
    print(f"Encontrados {len(product_elements)} elementos de produto para a marca '{marca_name}'")
    
    products = []
    for product in product_elements:
       
        
        json_data = product.find('textarea', class_='jsonGa')
        if json_data:
            json_data = json_data.text.strip()
            product_info = json.loads(json_data)
        else:
            print("Elemento jsonGa não encontrado.")
            continue
        
        name = product_info.get('name')
        link_element = product.find('meta', itemprop='url')
        link = link_element['content'] if link_element else None
        
        if not link:
            print(f"Link não encontrado para o produto: {name}")
            continue
        
        link = 'https://' + link
        print(f"Processando produto: {name}, URL: {link}")
        price = product_info.get('price')
        product_id = product_info.get('id')
        category = product_info.get('category')
        brand = product_info.get('brand')
        price_for_subs = product_info.get('priceForSubs')
        hide_subscriber_discount_price = product_info.get('hideSubscriberDiscountPrice')
        
        product_data = {
            "name": name,
            "link": link,
            "price": price,
            "id": product_id,
            "category": category,
            "brand": brand,
            "priceForSubs": price_for_subs,
            "hideSubscriberDiscountPrice": hide_subscriber_discount_price
        }
        
        # Obter detalhes adicionais do produto
        product_details, last_sku = extract_product_details(link, last_sku)
        if product_details:
            product_data.update(product_details)
        
        products.append(product_data)
        
        # Inserir o produto no arquivo produtos.json
        with open('produtos.json', 'r', encoding='utf-8') as f:
            produtos_data = json.load(f)
        
        # Verifica se a marca já existe no produtos_data
        existing_brand = next((m for m in produtos_data["marcas"] if m["name"] == marca_name), None)
        
        if existing_brand:
            existing_brand["produtos"].append(product_data)
        else:
            marca_data = {
                "name": marca_name,
                "link": marca_link,
                "produtos": [product_data]
            }
            produtos_data["marcas"].append(marca_data)
        
        # Salvar o arquivo produtos.json
        with open('produtos.json', 'w', encoding='utf-8') as f:
            json.dump(produtos_data, f, ensure_ascii=False, indent=4)
        
        print(f"Produto '{name}' adicionado à marca '{marca_name}'")
    
    return products, last_sku



def save_checkpoint(checkpoint_data):
    with open('checkpoint.json', 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, ensure_ascii=False, indent=4)
    print("Checkpoint salvo.")

def load_checkpoint():
    if os.path.exists('checkpoint.json'):
        with open('checkpoint.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    return {
        "last_sku": 0,
        "processed_brands": [],
        "total_products": 0,
        "added_products": [],
        "error_brands": []
    }



def process_brand(marca, last_sku, processed_brands, total_products, added_products, error_brands):
    url = marca['link']
    print(f"Processando marca: {marca['name']} - URL: {url}")
    try:
        html_content = get_html_with_requests(url)
    except Exception as e:
        print(f"Erro ao obter HTML para a marca {marca['name']} - URL: {url} - Erro: {e}")
        error_brands.append({"name": marca['name'], "link": url})
        save_checkpoint({"last_sku": last_sku, "processed_brands": processed_brands, "total_products": total_products, "added_products": added_products, "error_brands": error_brands})
        return False, last_sku, processed_brands, total_products, added_products, error_brands
    
    if html_content:
    
        products, last_sku = extract_products(html_content, marca['name'], url, last_sku)
        if products is None:
            error_brands.append({"name": marca['name'], "link": url})
            save_checkpoint({"last_sku": last_sku, "processed_brands": processed_brands, "total_products": total_products, "added_products": added_products, "error_brands": error_brands})
            return False, last_sku, processed_brands, total_products, added_products, error_brands
        print(f"{len(products)} produtos processados para a marca '{marca['name']}'")
        processed_brands.append(marca['name'])
        total_products += len(products)
        added_products.extend([product['name'] for product in products])
        save_checkpoint({"last_sku": last_sku, "processed_brands": processed_brands, "total_products": total_products, "added_products": added_products, "error_brands": error_brands})
    else:
        print(f"Falha ao obter HTML para a marca {marca['name']} - URL: {url}")
        error_brands.append({"name": marca['name'], "link": url})
        save_checkpoint({"last_sku": last_sku, "processed_brands": processed_brands, "total_products": total_products, "added_products": added_products, "error_brands": error_brands})
        return False, last_sku, processed_brands, total_products, added_products, error_brands
    
    return True, last_sku, processed_brands, total_products, added_products, error_brands

def main():
    print("Iniciando o processo principal")
    
    with open('marcas.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        print("Arquivo marcas.json carregado")
    
    if 'subcategories' not in data:
        print("Erro: A chave 'subcategories' não está presente no arquivo marcas.json")
        return
    
    # Criar o arquivo produtos.json se não existir
    if not os.path.exists('produtos.json'):
        with open('produtos.json', 'w', encoding='utf-8') as f:
            json.dump({"marcas": []}, f, ensure_ascii=False, indent=4)
        print("Arquivo produtos.json criado")
    
    # Adicionar as marcas ao arquivo produtos.json antes de processar os produtos
    with open('produtos.json', 'r', encoding='utf-8') as f:
        produtos_data = json.load(f)
    
    for marca in data['subcategories']:
        if not any(m["name"] == marca["name"] for m in produtos_data["marcas"]):
            produtos_data["marcas"].append({
                "name": marca["name"],
                "link": marca["link"],
                "produtos": []
            })
            print(f"Marca '{marca['name']}' adicionada ao arquivo produtos.json")
    
    with open('produtos.json', 'w', encoding='utf-8') as f:
        json.dump(produtos_data, f, ensure_ascii=False, indent=4)
    
    # Carregar checkpoint
    checkpoint = load_checkpoint()
    last_sku = checkpoint["last_sku"]
    processed_brands = checkpoint["processed_brands"]
    total_products = checkpoint["total_products"]
    added_products = checkpoint["added_products"]
    error_brands = checkpoint.get("error_brands", [])
    
    # Processar todas as marcas
    for marca in data['subcategories']:
        if marca['name'] not in processed_brands:
            _, last_sku, processed_brands, total_products, added_products, error_brands = process_brand(marca, last_sku, processed_brands, total_products, added_products, error_brands)
    
    print(f"Processo concluído. Total de produtos processados: {total_products}")
    print(f"Produtos adicionados: {added_products}")
    print(f"Marcas com erro: {error_brands}")

if __name__ == "__main__":
    main()
            

Iniciando o processo principal
Arquivo marcas.json carregado
Arquivo produtos.json criado
Marca '4 Groomer' adicionada ao arquivo produtos.json
Marca 'Acapulco Pet' adicionada ao arquivo produtos.json
Marca 'Açomix' adicionada ao arquivo produtos.json
Marca 'Acquapet' adicionada ao arquivo produtos.json
Marca 'Adaptil' adicionada ao arquivo produtos.json
Marca 'Adimax' adicionada ao arquivo produtos.json
Marca 'Advantage' adicionada ao arquivo produtos.json
Marca 'Advocate' adicionada ao arquivo produtos.json
Marca 'Afp' adicionada ao arquivo produtos.json
Marca 'Agachamber Vet' adicionada ao arquivo produtos.json
Marca 'Agener União' adicionada ao arquivo produtos.json
Marca 'AIC Soluções Veterinárias' adicionada ao arquivo produtos.json
Marca 'Akio' adicionada ao arquivo produtos.json
Marca 'Alambre' adicionada ao arquivo produtos.json
Marca 'Alcat' adicionada ao arquivo produtos.json
Marca 'Alcon Pet' adicionada ao arquivo produtos.json
Marca 'Alergoshop' adicionada ao arquivo produ